In [5]:
import numpy as np
import pydicom
from PIL import Image, ImageDraw

# your file paths
INPUT_DICOM_PATH = 'C:/MyPythonProjects/XipeAI/xipeai-frontend/uploads/extracted/0066/unnamed_8/IM-0024-0181.dcm'
MASK_PATH = ''
OUTPUT_DICOM_PATH = 'C:/Users/FDIFRIS/Downloads/test_dicom.dcm'

def apply_ct_window(img, window):
    # window = (window width, window level)
    R = (img-window[1]+0.5*window[0])/window[0]
    R[R<0] = 0
    R[R>1] = 1
    return R
  
ds = pydicom.dcmread(INPUT_DICOM_PATH)
img = ds.pixel_array # dtype = uint16
img = img.astype(float)
img = img*ds.RescaleSlope + ds.RescaleIntercept

display_img = apply_ct_window(img, [400,50])

# for this particular example
top, left, bottom, right = [211,99,291,158]
thickness = 4
img_bbox = Image.fromarray((255*display_img).astype('uint8'))
img_bbox = img_bbox.convert('RGB')
draw = ImageDraw.Draw(img_bbox)
for i in range(thickness):
    draw.rectangle(
        [left + i, top + i, right - i, bottom - i],
        outline=(255,0,0)
    )
del draw
img_bbox = np.asarray(img_bbox)

# modify DICOM tags
ds.PhotometricInterpretation = 'RGB'
ds.SamplesPerPixel = 3
ds.BitsAllocated = 8
ds.BitsStored = 8
ds.HighBit = 7
ds.add_new(0x00280006, 'US', 0)
ds.is_little_endian = True
ds.fix_meta_info()

# save pixel data and dicom file
ds.PixelData = img_bbox.tobytes()
ds.save_as(OUTPUT_DICOM_PATH)


In [3]:
# Example usage
png_path = 'C:/MyPythonProjects/XipeAI/xipeai-frontend/uploads/extracted/0066/unnamed_8/IM-0024-0181.dcm'
dcm_path = 'C:/Users/FDIFRIS/Downloads/test_dicom.dcm'
png_to_dcm(png_path, dcm_path)

DICOM file saved to C:/Users/FDIFRIS/Downloads/my-dicom-image(3).dcm


In [6]:
import numpy as np
import pydicom
from PIL import Image, ImageDraw
from skimage.measure import regionprops, label

# your file paths
INPUT_DICOM_PATH = 'C:/MyPythonProjects/XipeAI/xipeai-frontend/uploads/extracted/0066/unnamed_8/IM-0024-0181.dcm'
MASK_DICOM_PATH = 'C:/MyPythonProjects/XipeAI/xipeai-frontend/uploads/segmented_dicom/0066/unnamed_8/slice0181.dcm'  # update this with your actual mask DICOM path
OUTPUT_DICOM_PATH = 'C:/Users/FDIFRIS/Downloads/test_dicom.dcm'

def apply_ct_window(img, window):
    # window = (window width, window level)
    R = (img-window[1]+0.5*window[0])/window[0]
    R[R<0] = 0
    R[R>1] = 1
    return R

def get_bounding_boxes(mask, label_value):
    labeled_mask = label(mask == label_value)
    regions = regionprops(labeled_mask)
    boxes = []
    for region in regions:
        min_row, min_col, max_row, max_col = region.bbox
        boxes.append((min_row, min_col, max_row, max_col))
    return boxes

def draw_bounding_boxes(image, boxes, margin=5, thickness=4):
    img_bbox = Image.fromarray((255 * image).astype('uint8')).convert('RGB')
    draw = ImageDraw.Draw(img_bbox)
    for box in boxes:
        top, left, bottom, right = box
        top = max(0, top - margin)
        left = max(0, left - margin)
        bottom = min(image.shape[0], bottom + margin)
        right = min(image.shape[1], right + margin)
        for i in range(thickness):
            draw.rectangle(
                [left + i, top + i, right - i, bottom - i],
                outline=(255, 0, 0)
            )
    del draw
    return np.asarray(img_bbox)

# Load the DICOM image
ds = pydicom.dcmread(INPUT_DICOM_PATH)
img = ds.pixel_array.astype(float)
img = img * ds.RescaleSlope + ds.RescaleIntercept

# Apply the CT window
display_img = apply_ct_window(img, [400, 50])

# Load the mask DICOM
mask_ds = pydicom.dcmread(MASK_DICOM_PATH)
mask = mask_ds.pixel_array

# Get bounding boxes for regions labeled as 2
boxes = get_bounding_boxes(mask, 2)

# Draw bounding boxes on the image
img_bbox = draw_bounding_boxes(display_img, boxes)

# Modify DICOM tags
ds.PhotometricInterpretation = 'RGB'
ds.SamplesPerPixel = 3
ds.BitsAllocated = 8
ds.BitsStored = 8
ds.HighBit = 7
ds.add_new(0x00280006, 'US', 0)
ds.is_little_endian = True
ds.fix_meta_info()

# Save pixel data and DICOM file
ds.PixelData = img_bbox.tobytes()
ds.save_as(OUTPUT_DICOM_PATH)